In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json

import lightgbm as lgb
from sklearn.model_selection import StratifiedGroupKFold
print("Lightgbm version:", lgb.__version__)

import optuna
print("Optuna version:", optuna.__version__)

# local modules
import sys
sys.path.append("../src")
from preproc import process_train_data

Lightgbm version: 4.5.0
Optuna version: 4.0.0


/Users/mavillan/Library/Caches/pypoetry/virtualenvs/mcts-strength-variants-E8z0EJ47-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


***
### load and preprocess data

In [2]:
# define some paths
path_raw = Path("../data/raw")
path_processed = Path("../data/processed")
path_results = Path("../data/results")

# load data
df_train = pd.read_csv(path_raw / "train.csv")
df_test = pd.read_csv(path_raw / "test.csv")

df_train

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


In [3]:
# Call the function
df_train, numerical_cols, categorical_cols, encoder, scaler = process_train_data(
    df_train,
    scale=False,
)

# Print the results
print("Numerical Columns:", len(numerical_cols))
print("Categorical Columns:", len(categorical_cols))

number of all nan cols:  18
number of constant cols:  198
Numerical Columns: 588
Categorical Columns: 10


/Users/mavillan/repos/kg/mcts-strength-variants/feat_selection/../src/preproc.py:62: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_train["utility_agent1_rank"] = (


***
### optimization



In [4]:
# Define the number of folds for cross-validation
num_folds = 5

# Define the column for stratified or group k-fold
group_col = "GameRulesetName"
y_col = "utility_agent1_rank"
gkf = StratifiedGroupKFold(n_splits=num_folds, random_state=2113, shuffle=True)
split_list = list(gkf.split(df_train, groups=df_train[group_col], y=df_train[y_col]))


/Users/mavillan/Library/Caches/pypoetry/virtualenvs/mcts-strength-variants-E8z0EJ47-py3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:994: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [5]:
def train_and_score(numerical_cols, categorical_cols):

    # Define the parameters
    params = {
        'objective': "regression",
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'max_depth': 10,
        'learning_rate': 0.1,
        'reg_alpha': 1e-1,
        'reg_lambda': 1e-1,
        'feature_fraction': 0.8,
        'verbose': -1,
        'seed': 2112,
    }

    target = 'utility_agent1'
    oof_scores = []

    # Perform cross-validation
    for _, (train_index, val_index) in enumerate(split_list, 1):
        # Split the data
        X_train, X_val = df_train.iloc[train_index], df_train.iloc[val_index]
        y_train, y_val = X_train[target], X_val[target]
        
        # Create LightGBM datasets
        train_data = lgb.Dataset(
            data=X_train[numerical_cols + categorical_cols],
            label=y_train,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )
        val_data = lgb.Dataset(
            data=X_val[numerical_cols + categorical_cols],
            label=y_val,
            categorical_feature=categorical_cols,
            free_raw_data=True
        )

        # Train the model
        model = lgb.train(
            params,
            train_data,
            num_boost_round=1000,
            valid_sets=[val_data],
        )

        # predict on validation set
        y_pred = model.predict(X_val[numerical_cols + categorical_cols])
        y_pred = np.clip(y_pred, -1, 1)
        
        # Compute RMSE on scaled values
        rmse = np.sqrt(np.mean((y_pred - y_val) ** 2))
        oof_scores.append(rmse)

    return np.mean(oof_scores)


In [6]:
def objective(trial):
    # Feature selection
    selected_numerical = [col for col in numerical_cols if trial.suggest_int(f'use_{col}', 0, 1) == 1]
    selected_categorical = [col for col in categorical_cols if trial.suggest_int(f'use_{col}', 0, 1) == 1]

    if len(selected_numerical) + len(selected_categorical) == 0:
        return 1.
        
    # Call the train_and_score function with selected features
    score = train_and_score(selected_numerical, selected_categorical)
    return score

In [7]:
do_optimize = False


timeout = 3600 * 72

# Start with QMC for good coverage
study = optuna.create_study(
    study_name="optuna_lgbm",
    direction='minimize',
    storage='sqlite:///optuna_lgbm.db',
    load_if_exists=True,
    sampler=optuna.samplers.QMCSampler()
)

if do_optimize:
    study.optimize(objective, n_trials=100)

    # Switch to TPE for refined search
    study = optuna.create_study(
        study_name="optuna_lgbm",
        direction='minimize', 
        storage='sqlite:///optuna_lgbm.db',
        load_if_exists=True,
        sampler=optuna.samplers.TPESampler(
            n_startup_trials=100,  # Increase random sampling at start
            n_ei_candidates=100,   # Consider more candidates
            multivariate=True,     # Enable multivariate sampling
            constant_liar=True     # Help with parallel optimization
    )
    )
    study.optimize(objective, n_trials=10_000)

/var/folders/jw/hkbjn4190p76_0dtnrpy7zwm0000gn/T/ipykernel_33574/2723040410.py:12: ExperimentalWarning: QMCSampler is experimental (supported from v3.0.0). The interface can change in the future.
  sampler=optuna.samplers.QMCSampler()
[I 2024-11-03 14:12:05,049] Using an existing study with name 'optuna_lgbm' instead of creating a new one.


In [8]:
study.trials_dataframe().sort_values("value", ascending=True).head(20)

,number,value,datetime_start,datetime_complete,duration,params_use_Absolute,params_use_AbsoluteDirections,params_use_AddDecision,params_use_AddDecisionFrequency,params_use_AddEffect,...,params_use_agent1_exploration_const,params_use_agent1_playout,params_use_agent1_score_bounds,params_use_agent1_selection,params_use_agent2,params_use_agent2_exploration_const,params_use_agent2_playout,params_use_agent2_score_bounds,params_use_agent2_selection,state
2088,2088,0.427424,2024-11-03 12:35:59.643721,2024-11-03 12:37:08.983487,0 days 00:01:09.339766,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,COMPLETE
1539,1539,0.428311,2024-11-03 02:20:46.218649,2024-11-03 02:21:52.062007,0 days 00:01:05.843358,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,COMPLETE
1282,1282,0.428503,2024-11-02 21:44:28.522237,2024-11-02 21:45:34.726311,0 days 00:01:06.204074,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,COMPLETE
743,743,0.428517,2024-11-02 12:25:34.123510,2024-11-02 12:26:34.112969,0 days 00:00:59.989459,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,COMPLETE
1828,1828,0.428609,2024-11-03 07:41:11.921379,2024-11-03 07:42:20.350442,0 days 00:01:08.429063,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,COMPLETE
1418,1418,0.428614,2024-11-03 00:10:39.121276,2024-11-03 00:11:42.843129,0 days 00:01:03.721853,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,COMPLETE
1183,1183,0.428632,2024-11-02 19:59:21.288137,2024-11-02 20:00:24.956012,0 days 00:01:03.667875,1.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,COMPLETE
1040,1040,0.428785,2024-11-02 17:29:39.359807,2024-11-02 17:30:42.279756,0 days 00:01:02.919949,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,COMPLETE
1543,1543,0.429013,2024-11-03 02:25:08.001270,2024-11-03 02:26:13.509588,0 days 00:01:05.508318,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,COMPLETE
1245,1245,0.429161,2024-11-02 21:05:01.675882,2024-11-02 21:06:05.903822,0 days 00:01:04.227940,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,COMPLETE


In [9]:
# Get the selected features from the best trial
best_params = study.best_trial.params

# Filter numerical and categorical features
selected_numerical = [col for col in numerical_cols if best_params.get(f'use_{col}', 0) == 1]
selected_categorical = [col for col in categorical_cols if best_params.get(f'use_{col}', 0) == 1]

# Combine selected features
selected_features = selected_numerical + selected_categorical

print("Selected features:")
print("len(numerical): ", len(selected_numerical), "/", len(numerical_cols))
print("len(categorical): ", len(selected_categorical), "/", len(categorical_cols))
print("len(selected_features): ", len(selected_features), "/", len(numerical_cols) + len(categorical_cols))


# Save selected features to a file
selected_features_dict = {
    "numerical": selected_numerical,
    "categorical": selected_categorical
}
with open('select_optuna_lgbm.json', 'w') as f:
    json.dump(selected_features_dict, f, indent=2)

print("Selected features have been saved to 'select_optuna_lgbm.json'")

Selected features:
len(numerical):  296 / 588
len(categorical):  5 / 10
len(selected_features):  301 / 598
Selected features have been saved to 'select_optuna_lgbm.json'


***